In [ ]:
# TODO this is outdated and needs fixing

# Data in TINC

In [1]:
from tinc import *
tclient = TincClient()

# Usage of DiskBufferWriter

In [5]:
background = DiskBuffer('background', 'json', 'background.json', 'bin/')
tclient.register_disk_buffer(background)

AttributeError: 'DiskBuffer' object has no attribute 'Is'

In [15]:
import random, math

background.data = [random.random() for i in range(3)]

# Cache to compensate for possible time writing overlaps

In [16]:
dataWriter = DiskBufferWriter('sine_data','output.json', 'bin/')
dataWriter.expose_to_network(pserver)
dataWriter.enable_round_robin(10)

In [17]:
import random, math

numpoints = random.randint(4,20)

rnums = [random.random() for i in range(15)]
rnums.sort()
dataWriter.data = {'random' : [-1.0 + (n* 2.0) for n in rnums],
                   'sines' : [math.sin(r* 2 * math.pi) for r in rnums]
                  }

In [18]:
dataWriter.data

{'random': [-0.991967206140917,
  -0.8917266798894159,
  -0.7819396739248883,
  -0.7286422735256941,
  -0.7127629953701178,
  -0.5563408640092402,
  -0.5363229097607982,
  -0.4721036546227386,
  -0.2661573548321656,
  -0.16804753824757213,
  0.16523261747927087,
  0.20797501452650824,
  0.5028219360481028,
  0.6830976090166336,
  0.8411478250804942],
 'sines': [0.025233087720183695,
  0.3336291304092406,
  0.6327169343366218,
  0.7529250125703962,
  0.7848051524130282,
  0.9843763455796072,
  0.9934963120068258,
  0.9961621642432366,
  0.7420732320099696,
  0.5037522193607783,
  -0.49609332274725804,
  -0.6078679721965934,
  -0.9999607028332225,
  -0.8390736702576778,
  -0.4785905879048926]}

In [22]:
import time
for i in range(20):
    numpoints = random.randint(4,20)

    rnums = [random.random() for i in range(15)]
    rnums.sort()
    dataWriter.data = {'random' : [-1.0 + (n* 2.0) for n in rnums],
                       'sines' : [math.sin(r* 2 * math.pi) for r in rnums]
                      }
    time.sleep(0.1)

Now let's send changes really fast to test disk caching. (It may fail on your system if your drive access is not fast enough, causing the app to crash)

In [23]:
import time
dataWriter.enable_round_robin(100)

for i in range(200):
    numpoints = random.randint(4,20)

    rnums = [random.random() for i in range(15)]
    rnums.sort()
    dataWriter.data = {'random' : [-1.0 + (n* 2.0) for n in rnums],
                       'sines' : [math.sin(r* 2 * math.pi) for r in rnums]
                      }
    time.sleep(0.001)

This might leave a large cache, so cleanup:

In [30]:
dataWriter.cleanup_round_robin_files()

If you need guaranteed safety, use file locking:

In [31]:

# No cache but file locking
dataWriter.enable_round_robin(-1)
dataWriter.use_file_lock()

import time
for i in range(10000):
    numpoints = random.randint(4,20)

    rnums = [random.random() for i in range(15)]
    rnums.sort()
    dataWriter.data = {'random' : [-1.0 + (n* 2.0) for n in rnums],
                       'sines' : [math.sin(r* 2 * math.pi) for r in rnums]
                      }
    time.sleep(0.00001) # Very fast updates

If this is still not enough, you can have the data writer wait for read acknowledge from app.

Currently not implemented.

In [ ]:
pserver.stop()